# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Для начала давайте выдвенем некоторые гипотезы. Зависит ли семейное положение на факт погашения кредита в срок? Скорее да, чем нет. Чаще всего, когда человек находится в браке у него существует такое понятие как "семейный/общий бюджет" откуда можно взять часть денег для погашения кредита в случае надобности, так же само решение узаконить отношение достаточно серьезное и важное, так же как и взять кредит. Нужно подходить к этому разумно и просчитывать риски. Так что, если человек смог решится на такой шаг и нести отвественность перед своим партнером в отношениях, то скорее всего сможет осозновать всю серьезность положения кредитора. 
Что говорить о количестве детей, тут уже сложнее. С одной стороны, когда у тебя есть ребенок это дополнительная отвественность, ты не захочешь влезать в долги и подвергать риску свою семью, так что люди с детьми должны иметь меньше задолженностей, чем остальные, но с другой стороны - дети полны сюрпризов и очень часто происходят незапланированные траты. Например, новая одежда, детский сад, день рождения, да и деньги на новые шторы в школу. Так что тут тоже осмелюсь предположить, что люди, у которых нет детей будут более прилежными кредиторами и иметь меньше задолженностей.

## Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [ ]:
import pandas as pd #импортируем библиотеку Pandas
data=pd.read_csv('/datasets/data.csv') #сохраняем данные из файл data.csv в переменную data
data.info()#выводим общую информацию о содержании в файла на экран
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

В таблице содержется множество данных разных типов. Мы в основном будем работать со следующими столбцами *family_status*,*debt*,*total_income*, хотя другие стобцы тоже важны для анализа. Но для этого их нужно переработать: заполнить пропуски, избавиться от дубликатов и т.д.

## Шаг 2. Предобработка данных

### Первичная обработка данных

Посмотрев на данные можно сразу заметить несколько не очень хороших моментов. Например в столбце *education* по разному написано "Среднее" "среднее" "СРЕДНЕЕ", чтобы методы, которые мы будем применять к данному столбцу работали корректно, применим метод *str.lower()* к ячейкам столбца *education* 

In [ ]:
data['education']=data['education'].str.lower()
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Так же у нас есть отрицательные значения в столбце со стажем. Это артефакты — значения, которые не отражают действительность, нужно исправить это недразумение. Такая ситуация могла возникнуть из-за неправильного извлечения данных или некорректной анкеты, которую заполняли потенциальные заемщики. Например им нужно было написать о себе небольшой текст и они писали "Мой стаж - 1984 дня" и данные извлекались с "-", что привело к таким проблемам в таблице. Эту проблему можно решить через модуль. 

In [ ]:
data['days_employed']=abs(data['days_employed'])
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Теперь, кажется, можно приступать к обработке данных. Но! можно заметить в столбце *days_employed* критически большие, например, 340266.072047 , может быть это стаж не в днях, а в часах или других еденицах, если предположить, что это значение в часах, где каждый день работы считается как 24 часа стажа, то тогда это получается более реалистичное значение - 38 лет, но все еще очень неточное. Логичнее же брать рабочий день по 12 часов, тогда получается стаж 76 лет у человека 53 лет. Может быть это минуты, но тогда тоже что-то не сходится всего 1,2 года стажа. Это важный показатель, но мы не можем точно сказать чем вызвано появления таких больших значений в данных. Чтобы избежать проблем с такими крупными числами можно заменить их на медианное значение. Так же может в таблице встречаться такое, что стаж будет больше возраста человека или больше того времени, что он мог теоритически проработать. Человек не может быть официально трудоустроен до 18 лет. Так что можно заменять все значения, что больше ('dob_years'- 18) * 365  на медиану( это кажется логичным, но может быть я не права, пожалуйста дайте комментарий по этому вопросу)
Я считаю, что важно обработать данные в этом столбце, потому что стаж - отличный показатель стаблиности и трудоспособности кредитора.

In [ ]:
median_days_employed=data['days_employed'].median()
data.loc[data['days_employed'] > (data['dob_years']-18)*365,'days_employed' ] = median_days_employed
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,2194.220567,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,2194.220567,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Обработка пропусков

In [ ]:
print(data.isnull().sum())
print(data[['education','total_income']].head(15))


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
              education   total_income
0                высшее  253875.639453
1               среднее  112080.014102
2               среднее  145885.952297
3               среднее  267628.550329
4               среднее  158616.077870
5                высшее  255763.565419
6                высшее  240525.971920
7               среднее  135823.934197
8                высшее   95856.832424
9               среднее  144425.938277
10               высшее  113943.491460
11              среднее   77069.234271
12              среднее            NaN
13  неоконченное высшее  130458.228857
14               высшее  165127.911772


Проверили количество пропусков в столбцах таблицы и их обнаружилось достаточно много, примерно 10% сторк в каждом из двух столбцов утеряны, нужно попытаться восстановить эти данные. Пропуски присутсвуют в столбце с месячным доходом, это плохо, но не критично. Все можно исправить. Пропуски могли появиться из-за неверной транспортировки данных, так же может быть кто-то решил, что сможет получить кредит, не указывая свой доход, или данные были введены в неверном формате, напрмер, буквами - "двадцать пять тысяч". Это количественная переменная, такие пропуски следует заполнять характерными значениями. Среднее значение может быть не совсем показательным, ведь кредит могут брать как люди с очень маленьким, так и с достаточно большим заработком. Эти критически большие/малые значения могут "подпортить" среднее значение, поэтому среднее брать не стоит. Медианное слишком усреднит статистику в этом очень важном столбце. Поэтому можно присваивать какое-то усредненное значение в зависимости от категории. Зароботок зависит от образования(ну хотелось бы в это верить) и опыта работы. Посмотрим равномерно ли распределяются люди в таблице по уровню образования, что они получили. 

In [ ]:
print(data.groupby('education').mean()['total_income'])

education
высшее                 207142.515219
начальное              132155.513626
неоконченное высшее    181534.022774
среднее                153715.643971
ученая степень         174750.155792
Name: total_income, dtype: float64


Можно заметить, что в среднем самую высокую зарплату получают люди с высшим образованием, а самую низкую - с начальным. Что ж, это логично. Интересно, что люди с ученой степенью имеют меньший заработок, чем люди просто с высшим образованием. Скорее всего это из-за того, что это более узкие специалисты или ученые. 
Заполним пропуски в столбце 'total_income' средними значениями в соотвествии с категорией образования. 

In [ ]:
for ed in data['education'].unique():
    median=data.loc[data['education']==ed,'total_income'].median()
    data.loc[(data['total_income'].isna())&(data['education']==ed),'total_income']=median
print(data[['education','total_income']].head(15))    
print(data.isnull().sum())

              education   total_income
0                высшее  253875.639453
1               среднее  112080.014102
2               среднее  145885.952297
3               среднее  267628.550329
4               среднее  158616.077870
5                высшее  255763.565419
6                высшее  240525.971920
7               среднее  135823.934197
8                высшее   95856.832424
9               среднее  144425.938277
10               высшее  113943.491460
11              среднее   77069.234271
12              среднее  136478.643244
13  неоконченное высшее  130458.228857
14               высшее  165127.911772
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


Теперь заполним пропуски в столбцах со стажем. Со стажем сложнее определить, чем заполнять пустые ячейки. Но основным критерием должен быть возраст. Тут будем рассуждать так же, как и при изменении слишком больших значений в п.1.2.1. То есть пустые ячейки заменим числом ('dob_years'- 18) * 365.

In [ ]:
data.loc[(data['days_employed'].isna()),'days_employed']=(data['dob_years']-18)*365
print(data[['days_employed','dob_years']].tail(20))

       days_employed  dob_years
21505    2194.220567         53
21506    1556.249906         33
21507      79.832064         32
21508    2194.220567         62
21509    2194.220567         59
21510    3650.000000         28
21511     612.569129         29
21512     165.377752         26
21513    1166.216789         35
21514     280.469996         27
21515     467.685130         28
21516     914.391429         42
21517     404.679034         42
21518    2194.220567         59
21519    2351.431934         37
21520    4529.316663         43
21521    2194.220567         67
21522    2113.346888         38
21523    3112.481705         38
21524    1984.507589         40


#### Вывод

Мы заполнили пропуски в столбцах характерными значениями, зависящими от показателей в других столбцах. Теперь можно будет корректно работать с данными в этих столбцах.

### Замена типа данных

Необходимо заменить тип данных в столбцах со стажем и заработком для наглядности. Это упростит работу с этими столбцами.

In [ ]:
data.rename(columns={ data.columns[1]: "years_employed" },inplace=True)
data['years_employed']=data['years_employed']/365#переводим стаж в года
data['years_employed'] = data['years_employed'].astype('int')#делаем столбец days_employed int
#используем метод astype('int'), тк переводим числовой тип в другой числовой тип
#если бы нужно было переводить строку в число или число в дату, то использовались бы другие методы
data['total_income'] = data['total_income'].astype('int')#делаем столбец total_income int
data

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,6,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,6,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,12,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,6,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


#### Вывод

Заменили тип данных в строках *days_employed* и *total_income* на целочисленный. Сделали мы это, чтобы упростить работу с этими столбцами, все эти числа после точки очень усложняли чтение данных и не несли очень много информации, так же стаж перевели в года, это более привычный формат.

### Обработка дубликатов

In [ ]:
print(data.duplicated().sum())#смотрим количество дубликатов по всему датасету - их 71
data=data.drop_duplicates()#удаляем дубликаты
print(data.duplicated().sum())#проверяем удалились ли дубликаты
data

71
0


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,6,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,6,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,12,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,6,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,5,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,8,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


#### Вывод

Дубликаты могли появиться из-за повторного ввнесения данных в базу и после заполнения строк с пропущенными значениями, удаление дубликатов не повлияет на общую статистку, тк данные в строках совпдают это может не только усложнить работу с данными, но и повлиять на достоверность исследования.

### Лемматизация

In [ ]:
data.groupby('purpose').count()
#смотрим какие цели получения кредита есть в нашей таблицы и без труда замечаем,
#как некоторые из них похожи

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income
purpose,,,,,,,,,,,
автомобили,478,478,478,478,478,478,478,478,478,478,478
автомобиль,494,494,494,494,494,494,494,494,494,494,494
высшее образование,452,452,452,452,452,452,452,452,452,452,452
дополнительное образование,460,460,460,460,460,460,460,460,460,460,460
жилье,646,646,646,646,646,646,646,646,646,646,646
заняться высшим образованием,496,496,496,496,496,496,496,496,496,496,496
заняться образованием,408,408,408,408,408,408,408,408,408,408,408
на покупку автомобиля,471,471,471,471,471,471,471,471,471,471,471
на покупку подержанного автомобиля,478,478,478,478,478,478,478,478,478,478,478


Несложно заменить как много похожих, но по-разному сформулированных целей указано в столбце *purpose*, например, "строительство недвижимости", "стротельство собственной недвижимости","недвижимость", это все можно отнести к одной категории - "недвижимость". Этим и займемся.

In [ ]:
from pymystem3 import Mystem
m = Mystem() 
def lemmatize(row):#функция для замены всех схожих целей на одну единую
    lemmas=m.lemmatize(row)
    for words in lemmas:
        if ('недвижимость' in lemmas) or ('жилье' in lemmas):
            return 'недвижимость'
        if 'автомобиль' in lemmas:
            return 'автомобиль'
        if 'свадьба' in lemmas:
            return 'свадьба'
        if 'образование' in lemmas:
            return 'образование'
data=data.copy()
data['lemma_purpose'] = data['purpose'].apply(lemmatize)#добавление столбца с новыми названиеями целей
data=data.drop('purpose',1)#удаление старого столбца
data.head(30)       
    

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,lemma_purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,6,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,6,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


#### Вывод

После лемматизации гораздо проще группировать данные по столбцу *purpose*, теперь вместо 38 разных целей осталось лишь 4.

### Категоризация данных

Давайте попрробуем разделить наших кредиторов по группам по их заработку.

Теперь разделим людей по группам в соответвии их уровню дохода. Найдем минимальный, максимальный и средний доход, они соотвественно равны:

In [ ]:
print(data['total_income'].min())
print(data['total_income'].max())
print(round(data['total_income'].mean(),2))

20667
2265604
165412.96


Разделим людей на три группы: ниже среднего, средний, выше среднего. Верхняя граница для категори "ниже среднего": mean-(mean-min)/2=93040.

In [ ]:
def group_by_income(total_income):
    if(total_income>=165412):
        res='выше среднего'
    if((total_income>93040) & (total_income<165412)):
        res='средний'
    if(total_income<=93040):
        res='ниже среднего'
    return res
data['category_income']=data['total_income'].apply(group_by_income)
print(data.groupby('category_income').count()['debt'])#количество людей в каждой группе
print(data.groupby('category_income').mean()['debt'])


category_income
выше среднего    8197
ниже среднего    3670
средний          9587
Name: debt, dtype: int64
category_income
выше среднего    0.075393
ниже среднего    0.079292
средний          0.086784
Name: debt, dtype: float64


Люди с заработком "выше среднего" и "ниже среднего" чаще имеют проблемы с возвращением кредита в срок, по сравнению с людьми со "средним" заработком. стоит так же заметить, что в категории "ниже среднего" находится наименьшее количество людей. То есть из всей выборки большая часть имеет средний доход и выше.

In [ ]:
print(data.groupby('education').mean()['debt'])
print(data.groupby('gender').mean()['debt'])
print(data.groupby('income_type').mean()['debt'])

education
высшее                 0.052952
начальное              0.109929
неоконченное высшее    0.091398
среднее                0.089902
ученая степень         0.000000
Name: debt, dtype: float64
gender
F      0.070128
M      0.102624
XNA    0.000000
Name: debt, dtype: float64
income_type
безработный        0.500000
в декрете          1.000000
госслужащий        0.059025
компаньон          0.074045
пенсионер          0.056412
предприниматель    0.000000
сотрудник          0.095724
студент            0.000000
Name: debt, dtype: float64


#### Вывод

Посмотрев на статистику по разным группам людей, можно сказать, что лучше всего кредиты отдают девушки с заработком выше среднего работающий госслужащим с высшим образованием, а лучше даже, ученой степенью.

## Шаг 3. Ответьте на вопросы

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data.groupby('children').count()['debt']

children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

Если сгруппировать данные по количеству детей и посчитать сколько человек в каждой группе, то можно заметить, что больше половины кредиторов(14091) не имеют детей, далее 1/5 имеет по одному ребенку, и еще 8% имеет по два ребенка, еще меньше имеет по 4,5 детей. Так же в данных имееются такие аномалии, как -1 ребенок и 20 детей, тк количество кредиторов с таким количеством мало, мы не будем ориентироваьтся на эти показатели. Итак, если посмотреть, в каждой категории среднее значение по наличию/отсутсвию задолженностей, то можно заметить следующее

In [ ]:
data.groupby('children').mean()['debt']

children
-1     0.021277
 0     0.075438
 1     0.092346
 2     0.094542
 3     0.081818
 4     0.097561
 5     0.000000
 20    0.105263
Name: debt, dtype: float64

 Люди без детей реже имеют задолженности, чем люди с детьми. Скорее всего это как раз таки связано с тем, что с появлением детей появляется больше незапланированных расходов  в виде лекарств, одежды, учебы и т.д. И чем больше детей, тем больше этих расходов, это отражается на возврате кредитов в срок. Только с семьи с 3 детьми немного выпадают из последовательности, но в целом, с увелечением количества детей увеличивается среднее число задолженностей.

#### Вывод

Существует зависимость между количеством детей и возвратом кредита в срок. Люди, у которых нет детей реже имеют задолженности, а среди тех, кто имеет детей наблюдается следующая ситуация: чем больше детей, тем чаще кредиторы имеют задоженности.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data.groupby('family_status').count()['debt']

family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
Name: debt, dtype: int64

Если посчитать сколько человек в каждой группе, соответсвующей семейному положению, то здесь распределение произошло более равномерно, чем с количеством детей, но больше всего людей в группе "женат/замужем". Это весьма ожидаемо, люди которые вступают в брак и планируют детей часто берут кредиты на приобретение собственной недвижимости или машины. Посмотрим как от семейного положения зависит наличие задолженностей по кредиту.

In [ ]:
data.groupby('family_status').mean()['debt']

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
Name: debt, dtype: float64

По таблице видно, что вдовы и вдовцы имеют самый лучший показатель по столбцу 'debt', далее почти одинаковые показатели у женатых/замужних людей и тех, кто в разводе. Это скорее всего связано с тем, что у тех, кто в браке, есть некоторые обязательства и отвественность перед членами семьи, а у тех кто в разводе могут быть дети и кредит, который они выплачивают идет на постройку им жилья или оплату образования. Хуже дела обстоят у тех, кто в гражданском браке, под этим скорее всего имеется в виду сожительство. Такие отношения не закреплены на бумаге, не имеют законного подтверждения и не обязывают ни к чему, то есть при любой проблеме ваш партнер может просто уйти, в этом скорее всего и кроется проблема. Если вы в гражданском браке, то если вы, например, берете кредит на недвижимость, то когда вы расстанетесь, ваш парнер не получит 50% от этой недвижимости, то есть и помогать вам с выплатой кредита ему смысла нет. Жестоко, но это жизнь. У тех кто не женат/не замужем думаю та же проблема. Одно дело бюджет двух людей, а другое одного. 

#### Вывод

Наблюдается некоторая зависимоть между семейным положением и возвратом кредита в срок. Люди находящиеся или находившиеся в узаконенных отношениях имеют меньше проблем с возвращением кредитов в срок по сравнению с теми, кто находится в гражданском браке или не находится в отношениях. Вывод лишь один - "человеку нужен человек", хотя бы чтобы вместе выплачивать кредит.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Ранее, в пункте 1.2.6. мы уже делили данные по уровню дохода. Давайте еще раз взглянем на получившееся распределение:


In [ ]:
def group_by_income(total_income):
    if(total_income>=165412):
        res='выше среднего'
    if((total_income>93040) & (total_income<165412)):
        res='средний'
    if(total_income<=93040):
        res='ниже среднего'
    return res
data['category_income']=data['total_income'].apply(group_by_income)
print(data.groupby('category_income').count()['debt'])#количество людей в каждой группе
print(data.groupby('category_income').mean()['debt'])


category_income
выше среднего    8197
ниже среднего    3670
средний          9587
Name: debt, dtype: int64
category_income
выше среднего    0.075393
ниже среднего    0.079292
средний          0.086784
Name: debt, dtype: float64


Можно попробовать разделить данные на еще более мелкие промежутки.

In [ ]:
def group_by_income(total_income):
    if(total_income>=1050096):
        res='очень высокий'
    if((total_income>697754) & (total_income<1050096)):
        res='высокий'
    if(total_income<=697754)&(total_income>165412):
        res='выше среднего'
    if(total_income>93040)&(total_income<=165412):
        res='средний'
    if(total_income<=93040)&(total_income>53858):
        res='ниже среднего'
    if(total_income<=53858):
        res='низкий'
    return res
data['category_income_small']=data['total_income'].apply(group_by_income)
print(data.groupby('category_income_small').count()['debt'])#количество людей в каждой группе
print(data.groupby('category_income_small').mean()['debt'])


category_income_small
высокий            40
выше среднего    8136
ниже среднего    3151
низкий            519
очень высокий      21
средний          9587
Name: debt, dtype: int64
category_income_small
высокий          0.100000
выше среднего    0.075344
ниже среднего    0.081561
низкий           0.065511
очень высокий    0.047619
средний          0.086784
Name: debt, dtype: float64


#### Вывод

Средний доход все еще имеет большенство людей из таблицы. Но если посмотреть как теперь распределся уровень задолженностей, то можно немного запутаться. Люди с "высоким" доходом имеют самый высокий показатель наличия задолженностей, в то время как люди с "очень высоким" имеют самый низкий. Люди с низким дохом стараются возвращать кредит в срок и получается это у них почти так же хорошо, как у людей с "очень высоким доходом". Дальше идут люди с доходом "выше среднего" , а после почти на равных "средний" и "ниже среднего". Достатоно сложно найти тут какую-то закономерность. Но думаю что-то тут все же есть. 
Люди, которые имеют высокий заработок имеют возможность и мотивацию стараются улучшить свое положение и берут кредиты, чтобы вложиться в свое образование или бизнес, но частенько прогорают и влезают в долги. Люди же с очень высоким зароботком имеют возможность выплачивать все вовремя и не хотят иметь дело с последствиями от несвоевременных выплат. Так же как и люди с низким заработком. Но это все в качестве предположения.

### - Как разные цели кредита влияют на его возврат в срок?

Сгруппируем данные по цели кредита и посмотим на количество людей в каждой группе и среднее значение по наличию задолженностей.

In [ ]:
print(data.groupby('lemma_purpose').count()['debt'])
print(data.groupby('lemma_purpose').mean()['debt'])


lemma_purpose
автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: debt, dtype: int64
lemma_purpose
автомобиль      0.093590
недвижимость    0.072334
образование     0.092200
свадьба         0.080034
Name: debt, dtype: float64


Больше всего людей берут кредит на недвижимость и эта категория имеет 7,2% должников, что являеется лучшим показателем среди других территорий. Это может быть связано с тем, что кредит на недвижимость часто берут люди, которые "уверенно стоят на ногах" и  имеют стабильный заработок. Чуть хуже дела обстоят с кредитами на свадьбу, но наибольший процент должников среди тех, кто берет кредиты на образование и автомобиль. Тут причина может быть в том, что эти кредиты чаще всего не на очень большие суммы и их могут дать людям и с не очень большим заработком и стажем, например. 

#### Вывод

Меньше всего должников среди тех, кто берет кредит на недвижимость, далее идут будущие молодожены, потом жаждущие знаний кредиторы и хуже всего те, кто берут кредиты на машины. Среди них аж 9,3% должников.

## Шаг 4. Общий вывод

Обработав данные, заполнив пропуски, удалив дубликаты, мы смогли проанализировать данные таблицы и получить весьма интересные результаты. Очень хорошо видна зависимоть между наличием детей и возвратом кредита в срок. Чем больше детей, тем больше долгов. Так же прослеживается зависимость между семейным положением и возвратом кредита в срок. Среди вдовцов меньше всего должников, дальше идут те, кто замужем или в разводе. И хуже всего себя показывают одинокие люди или те, кто состоят в гражданском браке. То есть можно сделать вывод, что лучшим заемщиком дл банка является человек, у которого нет детей, но он состоит в браке, или разведен, или вдовец. А худший вариант - это одинокий или состоящий в гражданском браке человек с несколькими детьми. К этому всему стоит добавить, что надежнее всего люди либо с очень высоким, либо с низким заработком, что удивительно, но факт. И конечно, стоит охотно давать кредиты на недвижимость, их чаще всего возвращают в срок. 
Следуя данным из таблицы можно сказать, кто является мечтой любого банка:
девушка очень высоким заработком, работающая госслужащей с высшим образованием, а лучше даже, ученой степенью, вдова, без детей, берущая кредит на недвижимость. 
Но где же такую найти? Этот вопрос оставим отделу по работе с клиентами. 




#буду рада любым комментариям и поправкам, прошу прощения за позднюю сдачу. готова к критике в любых ее проявлениях. Спасибо за уделенное время!

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.